In [ ]:
import '../../loadenv.mjs'

# Define State

In [2]:
import { END, Annotation } from '@langchain/langgraph'
import { BaseMessage } from '@langchain/core/messages'

const AgentState = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
        default: () => [],
    }),
    next: Annotation<string>({
        reducer: (x, y) => y ?? x ?? END,
        default: () => END,
    })
})

# Create tools

In [3]:
import { chartTool } from './../../utils.mjs'
import { TavilySearchResults } from '@langchain/community/tools/tavily_search'

const tavilyTool = new TavilySearchResults()

# Create Agent Supervisor

In [4]:
import { z } from 'zod'
import { getModel } from './../../utils.mjs'
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'
import { AIMessage } from '@langchain/core/messages'

const members = ['researcher', 'chart_generator'] as const
const systemPrompt = "You are a supervisor tasked with managing a conversation between the" +
  " following workers: {members}. Given the following user request," +
  " respond with the worker to act next. Each worker will perform a" +
  " task and respond with their results and status. When finished," +
  " respond with FINISH."
const options = [END, ...members]

const routingTool = {
    name: 'route',
    description: 'Select the next role.',
    schema: z.object({
        next: z.enum([END, ...members]),
    })
}

const prompt = ChatPromptTemplate.fromMessages([
    ['system', systemPrompt],
    new MessagesPlaceholder('messages'),
    [
        'human',
        'Given the conversation above, who should act next?' +
        ' Or should we FINISH? Select one of: {options}',
    ]
])

const formattedPrompt = await prompt.partial({
    options: options.join(', '),
    members: members.join(', '),
})

const llm = getModel()

const supervisorChain = formattedPrompt
    .pipe(llm.bindTools(
        [routingTool],
        {
            tool_choice: 'route',
        },
    ))
    .pipe((x: AIMessage) => x?.tool_calls?.[0]?.args)


In [ ]:
import { HumanMessage } from '@langchain/core/messages'

await supervisorChain.invoke({
    messages: [
        new HumanMessage({
            content: 'write a report on birds',
        })
    ]
})

# Construct Graph

In [16]:
import { RunnableConfig } from '@langchain/core/runnables'
import { createReactAgent } from '@langchain/langgraph/prebuilt'
import { SystemMessage } from '@langchain/core/messages'

const researcherAgent = createReactAgent({
    llm,
    tools: [tavilyTool],
    stateModifier: new SystemMessage("You are a web researcher. You may use the Tavily search engine to search the web for" +
    " important information, so the Chart Generator in your team can make useful plots.")
})

const researcherNode = async (
    state: typeof AgentState.State,
    config?: RunnableConfig,
) => {
    const result = await researcherAgent.invoke(state, config)
    const lastMessage = result.messages[result.messages.length - 1]
    return {
        messages: [
            new HumanMessage({
                content: lastMessage.content,
                name: 'Researcher',
            })
        ]
    }
}

const chartGenAgent = createReactAgent({
    llm,
    tools: [chartTool],
    stateModifier: new SystemMessage("You excel at generating bar charts. Use the researcher's information to generate the charts.\nIf you don't have necessary data to generate bar charts, you should ask supervisor for it")
})

const chartGenNode = async (
    state: typeof AgentState.State,
    config?: RunnableConfig,
) => {
    const result = await chartGenAgent.invoke(state, config)
    const lastMessage = result.messages[result.messages.length - 1]
    return {
        messages: [
            new HumanMessage({
                content: lastMessage.content,
                name: 'ChartGenerator',
            })
        ]
    }
}

In [ ]:
import { START, StateGraph } from '@langchain/langgraph'

const workflow = new StateGraph(AgentState)
    .addNode('researcher',researcherNode)
    .addNode('chart_generator', chartGenNode)
    .addNode('supervisor', supervisorChain)

members.forEach(member => {
    workflow.addEdge(member, 'supervisor')
})

workflow.addConditionalEdges(
    'supervisor',
    (x: typeof AgentState.State) => x.next,
)

workflow.addEdge(START, 'supervisor')

const graph = workflow.compile()

In [ ]:
import { printGraph } from './../../utils.mjs'

await printGraph(graph.getGraph())

# Invoke the team

In [ ]:
let streamResults = graph.stream(
    {
        messages: [
            new HumanMessage({
                content: 'What were the 3 most popular tv shows in 2023?',
            })
        ]
    },
    {
        recursionLimit: 100,
    }
)

for await (const output of await streamResults) {
    if (!output?.__end__) {
        console.log(output)
        console.log('----')
    }
}

In [ ]:
streamResults = graph.stream(
    {
        messages: [
            new HumanMessage({
                content: 'Generate a bar chart of the US GDP growth from 2021-2023',
            }),
        ],
    },
    {
        recursionLimit: 150,
    }
)

for await (const output of await streamResults) {
    if (!output?.__end__) {
        console.log(output)
        console.log('----')
    }
}